In [1]:
import pandas as pd
import simplejson

In [2]:
# Set file paths
base_path = "../data/"
book_file = "book-A-song-of-Ice-and-Fire.csv"
tv_file = "tv-Game-of-Thrones.csv"

In [3]:
#import csv files to dataframes
bookdf = pd.read_csv(base_path + book_file)
bookdf["category"] = "book"

tvdf = pd.read_csv(base_path + tv_file)
tvdf['category'] = "tv"

In [4]:
#read in lists of character names to lists
fb  = open("../data/book_characters.json", "r")
book_char_list = simplejson.loads(fb.read())

ft  = open("../data/tv_characters.json", "r")
tv_char_list = simplejson.loads(ft.read())

In [5]:
# One character as listed under two names. 
# Dropping this column because it had fewer enteires,
# all of which were redundant to the surviving columnn. 
bookdf = bookdf.drop('Aegon T./Egg', 1)
book_char_list.remove('Aegon T./Egg')

In [6]:
#rename some popular characters to thiers name are consistent across book/tv datasets.

def rename_longer(cb, ct):
    global tv_char_list
    global book_char_list
    if len(cb) > len(ct) and "Nymeria" not in cb:
        tvdf.rename(columns={ct: cb}, inplace=True)
        tv_char_list = [cb if x==ct else x for x in tv_char_list]
        print "Book: {}, TV: {} - Changed in TV.".format(cb, ct)
    elif "Nymeria" not in cb:
        bookdf.rename(columns={cb: ct}, inplace=True)
        book_char_list = [ct if x==cb else x for x in book_char_list]
        print "Book: {}, TV: {} - Changed in book.".format(cb, ct)
        
for cb in book_char_list:
    if cb not in tv_char_list:
        for ct in tv_char_list:
                if ct.split("/")[0] == cb:
                    rename_longer(cb, ct)
                elif cb.split("/")[0] == ct:
                    rename_longer(cb, ct)
                elif cb in ct:
                    rename_longer(cb, ct)
                elif ct in cb:
                    rename_longer(cb, ct)

rename_longer("Catelyn T.", "Catelyn S.")
rename_longer("Brienne T.", "Brienne of Tarth")

Book: Aemon T./The Dragonknight, TV: Aemon T. - Changed in TV.
Book: Brynden T., TV: Brynden T./Brynden the Blackfish - Changed in book.
Book: Daenerys T./Dany, TV: Daenerys T. - Changed in TV.
Book: Eddard S./Ned, TV: Eddard S. - Changed in TV.
Book: Gendry, TV: Gendry W. - Changed in book.
Book: Myranda R., TV: Myranda - Changed in TV.
Book: Petyr B./Littlefinger, TV: Petyr B. - Changed in TV.
Book: Rhaenys T./Queen Who Never Was, TV: Rhaenys T. - Changed in TV.
Book: Shaera T., TV: Shae - Changed in TV.
Book: Catelyn T., TV: Catelyn S. - Changed in book.
Book: Brienne T., TV: Brienne of Tarth - Changed in book.


In [7]:
# Merge tables using TV as the standard since it has less characters and only characters appearing in both will be popular enough to appear on the chart. 
# Check if headers only appearing in tv are important people (they should not be)
tv_not_book = [x for x in list(tvdf) if x not in list(bookdf)]
for i in tv_not_book:
    print "{} : tv-{}".format(i, tvdf[i].sum())

Direwolves : tv-38
Dragons : tv-12
G. M. Pycelle : tv-3
Grey Worm : tv-10
Karl T. : tv-1
Khal Drogo : tv-91
Locke : tv-2
Ludd W. : tv-1
M. Luwin : tv-5
Rodrick F. : tv-1
Royland D. : tv-1
Ryon F. : tv-1
S. Mordane : tv-1
Talia F. : tv-1
Talisa M. : tv-21
Viserion T. : tv-1
Yara G. : tv-28
the three eyed raven : tv-1


In [17]:
# Innerjoing the tables. Will result in droping above characters from tv dataset. 
gotdf = pd.concat([bookdf,tvdf], join="inner")
got_char_list = [x for x in tv_char_list if x in book_char_list]

#test and make sure it merged right
key_char = ["Jon S.", "Robb S.", "Sansa S.", "OC"]
for char in key_char:
    if not bookdf[char].sum() + tvdf[char].sum() == gotdf[char].sum():
        print "Join missing connections" 

In [11]:
#Create a list of dictionarys contaning links between characters and the rating for those links
output = []
for character in got_char_list:
    for name in got_char_list:
        if character is not name:
            stories = gotdf[(gotdf[character] == True) & (gotdf[name] == True)].count()['id']
            m_ratings = float(gotdf[(gotdf[character] == True) & (gotdf[name] == True) & (gotdf['rated'] == "M")].count()['id'])
            if stories > 0:
                m_ratio = m_ratings/stories
            else:
                m_ratio = 0
            output.append({"Character_A":character, "Character_B":name, "count":stories, "rating":m_ratio})

In [12]:
#sort the output by number of stories in a link
output.sort(key=lambda x:(x['count'], x['rating']))
output.reverse()

In [16]:
#write output the top 40 links to file
ft  = open("../data/got_vis_data.json", "w")
ft.write(simplejson.dumps(output[:40]))